In [ ]:
from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)
import logging
import time
import os

logging.basicConfig(level=logging.DEBUG)

email = os.getenv('GARMIN_EMAIL')
password = os.getenv('GARMIN_PASSWORD')

MAX_RETRIES = 3
RETRY_DELAY = 60  # seconds

def connect_with_retry():
    for attempt in range(MAX_RETRIES):
        try:
            client = Garmin(email, password)
            client.login()
            return client
            
        except GarminConnectTooManyRequestsError:
            if attempt < MAX_RETRIES - 1:
                print(f"Rate limited. Waiting {RETRY_DELAY} seconds before retry...")
                time.sleep(RETRY_DELAY)
                continue
            raise
            
        except (GarminConnectAuthenticationError, GarminConnectConnectionError) as err:
            print(f"Error during attempt {attempt + 1}: {err}")
            raise

try:
    client = connect_with_retry()
    print("Successfully connected to Garmin!")
    
    # Test the connection with a basic API call
    activities = client.get_activities(0, 1)
    if activities:
        print(f"Most recent activity: {activities[0]['activityName']}")

except GarminConnectTooManyRequestsError:
    print("Failed after maximum retries due to rate limiting. Please wait a few minutes before trying again.")
except GarminConnectAuthenticationError as err:
    print(f"Authentication error: {err}")
except GarminConnectConnectionError as err:
    print(f"Connection error: {err}")
except Exception as err:
    print(f"Unknown error occurred: {err}")
    print(f"Error type: {type(err)}")
    import traceback
    traceback.print_exc()